In [ ]:
"""
AVANCE1:

🌱 AGRO-AI VISION PRO - Sistema de Diagnóstico de Enfermedades en Plantas
==========================================================================
Sistema avanzado de detección de enfermedades con:
- Transfer Learning (MobileNetV2)
- Data Augmentation
- Visualizaciones profesionales
- Matriz de confusión
- Predicciones con probabilidades
- Exportación de reportes
- Interfaz mejorada para demo

Autor: M.I.N.D RESEARCH GROUP
Hackathon: [SAMSUMG INNOVATION CAMPUS]
"""

import os
import json
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import pathlib
from datetime import datetime

# ==========================================
# 🔧 CONFIGURACIÓN INICIAL
# ==========================================
print("="*60)
print("🌱 AGRO-AI VISION PRO - Inicializando Sistema")
print("="*60)

# Limpieza de configuración anterior
if os.path.exists('/root/.kaggle'):
    !rm -rf /root/.kaggle
    print("✅ Configuración anterior limpiada")

# Cargar credenciales de Kaggle
print("\n📥 Cargue su archivo kaggle.json:")
uploaded = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("✅ Credenciales de Kaggle configuradas")

# Descargar dataset
print("\n📦 Descargando dataset PlantVillage...")
!kaggle datasets download -d emmarex/plantdisease
!unzip -q plantdisease.zip
print("✅ Dataset descargado y descomprimido")

# ==========================================
# 📊 CONFIGURACIÓN DEL MODELO
# ==========================================
# Configuración optimizada para balance entre precisión y velocidad
BATCH_SIZE = 32
IMG_HEIGHT = 224  # Tamaño óptimo para MobileNetV2
IMG_WIDTH = 224
EPOCHS = 20
LEARNING_RATE = 0.001

# Verificar ruta del dataset
if os.path.exists('PlantVillage'):
    data_dir = pathlib.Path("PlantVillage")
    print("✅ Dataset PlantVillage encontrado")
else:
    data_dir = pathlib.Path(".")
    print("⚠️ Usando directorio actual")

# ==========================================
# 🎨 DATA AUGMENTATION AVANZADO
# ==========================================
print("\n🎨 Configurando Data Augmentation...")

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2),
])

# ==========================================
# 📂 CARGA DE DATOS
# ==========================================
print("\n📂 Cargando datos de entrenamiento...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

print("📂 Cargando datos de validación...")
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"\n✅ {num_classes} clases detectadas")
print(f"📋 Primeras 10 clases: {class_names[:10]}")

# Optimización de rendimiento
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ==========================================
# 🏗️ CONSTRUCCIÓN DEL MODELO CON TRANSFER LEARNING
# ==========================================
print("\n🏗️ Construyendo modelo con Transfer Learning (MobileNetV2)...")

def create_model(num_classes, img_height, img_width):
    """
    Crea un modelo de CNN usando Transfer Learning con MobileNetV2
    pre-entrenado en ImageNet
    """
    # Cargar modelo base pre-entrenado
    base_model = MobileNetV2(
        input_shape=(img_height, img_width, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Congelar las capas base inicialmente
    base_model.trainable = False
    
    # Construir el modelo completo
    inputs = tf.keras.Input(shape=(img_height, img_width, 3))
    
    # Data augmentation (solo en entrenamiento)
    x = data_augmentation(inputs)
    
    # Preprocesamiento específico de MobileNetV2
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    
    # Modelo base
    x = base_model(x, training=False)
    
    # Capas superiores personalizadas
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    
    # Capa de salida
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    
    return model, base_model

model, base_model = create_model(num_classes, IMG_HEIGHT, IMG_WIDTH)

# Compilar modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
)

print("\n📋 Arquitectura del Modelo:")
model.summary()

# ==========================================
# 🎯 CALLBACKS AVANZADOS
# ==========================================
print("\n🎯 Configurando callbacks...")

# Early Stopping: Detiene el entrenamiento si no hay mejora
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Reduce Learning Rate: Reduce LR cuando la métrica se estanca
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

# Model Checkpoint: Guarda el mejor modelo
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

callbacks = [early_stopping, reduce_lr, checkpoint]

# ==========================================
# 🚀 FASE 1: ENTRENAMIENTO INICIAL
# ==========================================
print("\n" + "="*60)
print("🚀 FASE 1: Entrenamiento con capas base congeladas")
print("="*60)

history_phase1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

# ==========================================
# 🔓 FASE 2: FINE-TUNING
# ==========================================
print("\n" + "="*60)
print("🔓 FASE 2: Fine-tuning - Descongelando capas superiores")
print("="*60)

# Descongelar las últimas 50 capas del modelo base
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

print(f"Capas entrenables: {len([l for l in model.layers if l.trainable])}")

# Recompilar con learning rate más bajo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
)

# Continuar entrenamiento
history_phase2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    initial_epoch=len(history_phase1.history['loss']),
    callbacks=callbacks,
    verbose=1
)

# Combinar historiales
history = {
    'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
    'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
    'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
    'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
}

print("\n✅ Entrenamiento completado!")

# ==========================================
# 📊 VISUALIZACIÓN DE RESULTADOS
# ==========================================
print("\n📊 Generando visualizaciones...")

def plot_training_history(history):
    """Visualiza el histórico de entrenamiento"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Accuracy
    axes[0, 0].plot(history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 0].plot(history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history['loss'], label='Train Loss', linewidth=2)
    axes[0, 1].plot(history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Accuracy comparison bar
    final_train_acc = history['accuracy'][-1]
    final_val_acc = history['val_accuracy'][-1]
    axes[1, 0].bar(['Train', 'Validation'], [final_train_acc, final_val_acc], 
                   color=['#2ecc71', '#3498db'], width=0.5)
    axes[1, 0].set_title('Final Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].set_ylim([0, 1])
    for i, v in enumerate([final_train_acc, final_val_acc]):
        axes[1, 0].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')
    
    # Metrics summary
    axes[1, 1].axis('off')
    metrics_text = f"""
    📊 RESUMEN DE MÉTRICAS
    {'='*40}
    
    🎯 Accuracy Final (Train): {final_train_acc:.4f}
    ✅ Accuracy Final (Val):   {final_val_acc:.4f}
    
    📉 Loss Final (Train):     {history['loss'][-1]:.4f}
    📉 Loss Final (Val):       {history['val_loss'][-1]:.4f}
    
    📈 Mejor Val Accuracy:     {max(history['val_accuracy']):.4f}
    📉 Mejor Val Loss:         {min(history['val_loss']):.4f}
    
    🔄 Épocas Entrenadas:      {len(history['accuracy'])}
    """
    axes[1, 1].text(0.1, 0.5, metrics_text, fontsize=11, 
                    family='monospace', verticalalignment='center')
    
    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Gráfico guardado como 'training_history.png'")

plot_training_history(history)

# ==========================================
# 🎯 MATRIZ DE CONFUSIÓN
# ==========================================
print("\n🎯 Generando matriz de confusión...")

def generate_confusion_matrix(model, val_ds, class_names):
    """Genera y visualiza la matriz de confusión"""
    y_true = []
    y_pred = []
    
    for images, labels in val_ds:
        predictions = model.predict(images, verbose=0)
        y_pred.extend(np.argmax(predictions, axis=1))
        y_true.extend(labels.numpy())
    
    # Calcular matriz de confusión
    cm = confusion_matrix(y_true, y_pred)
    
    # Si hay muchas clases, mostrar solo las 20 más frecuentes
    if len(class_names) > 20:
        top_classes_idx = np.argsort(np.sum(cm, axis=1))[-20:]
        cm = cm[top_classes_idx][:, top_classes_idx]
        selected_classes = [class_names[i] for i in top_classes_idx]
    else:
        selected_classes = class_names
    
    # Visualizar
    plt.figure(figsize=(16, 14))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=selected_classes, 
                yticklabels=selected_classes,
                cbar_kws={'label': 'Frecuencia'})
    plt.title('Matriz de Confusión - Top 20 Clases', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Predicción', fontsize=12)
    plt.ylabel('Real', fontsize=12)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✅ Matriz guardada como 'confusion_matrix.png'")
    
    # Reporte de clasificación
    print("\n📋 Reporte de Clasificación (Top 10 clases):")
    report = classification_report(y_true, y_pred, target_names=class_names, 
                                    output_dict=True, zero_division=0)
    
    # Mostrar las 10 mejores clases por F1-score
    class_scores = [(name, data['f1-score']) for name, data in report.items() 
                    if name not in ['accuracy', 'macro avg', 'weighted avg']]
    class_scores.sort(key=lambda x: x[1], reverse=True)
    
    print("\n🏆 Top 10 Clases por F1-Score:")
    for i, (class_name, score) in enumerate(class_scores[:10], 1):
        print(f"{i:2d}. {class_name:40s} F1: {score:.4f}")
    
    return cm, report

cm, report = generate_confusion_matrix(model, val_ds, class_names)

# ==========================================
# 🔍 SISTEMA DE PREDICCIÓN AVANZADO
# ==========================================
print("\n🔍 Configurando sistema de predicción...")

def diagnose_plant(model, image, class_names, top_k=5):
    """
    Realiza diagnóstico de una planta con probabilidades detalladas
    """
    # Preparar imagen
    img_array = tf.expand_dims(image, 0)
    
    # Hacer predicción
    predictions = model.predict(img_array, verbose=0)
    probabilities = tf.nn.softmax(predictions[0]).numpy()
    
    # Obtener top K predicciones
    top_indices = np.argsort(probabilities)[-top_k:][::-1]
    top_probs = probabilities[top_indices]
    top_classes = [class_names[i] for i in top_indices]
    
    return {
        'predicted_class': top_classes[0],
        'confidence': float(top_probs[0]),
        'top_predictions': list(zip(top_classes, [float(p) for p in top_probs]))
    }

def visualize_prediction(image, result, class_names):
    """Visualiza una predicción con estilo profesional"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Imagen
    ax1.imshow(image.astype("uint8"))
    ax1.axis('off')
    ax1.set_title(f'Muestra Analizada', fontsize=14, fontweight='bold')
    
    # Probabilidades
    classes, probs = zip(*result['top_predictions'])
    colors = plt.cm.RdYlGn(np.array(probs))
    y_pos = np.arange(len(classes))
    
    bars = ax2.barh(y_pos, probs, color=colors)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels([c.replace('___', ' - ').replace('_', ' ') for c in classes])
    ax2.set_xlabel('Probabilidad', fontsize=12)
    ax2.set_title('Top 5 Predicciones', fontsize=14, fontweight='bold')
    ax2.set_xlim([0, 1])
    
    # Añadir valores en las barras
    for i, (bar, prob) in enumerate(zip(bars, probs)):
        ax2.text(prob + 0.02, i, f'{prob:.2%}', 
                va='center', fontweight='bold')
    
    # Añadir grid
    ax2.grid(axis='x', alpha=0.3)
    ax2.invert_yaxis()
    
    plt.tight_layout()
    return fig

# ==========================================
# 🎬 DEMO: DIAGNÓSTICO EN TIEMPO REAL
# ==========================================
print("\n" + "="*60)
print("🎬 DEMO: Sistema de Diagnóstico en Tiempo Real")
print("="*60)

# Tomar múltiples muestras para demo
print("\n🔬 Analizando 6 muestras aleatorias del conjunto de validación...")

fig, axes = plt.subplots(3, 2, figsize=(16, 20))
axes = axes.ravel()

image_batch, label_batch = next(iter(val_ds))

for idx in range(6):
    image = image_batch[idx].numpy().astype("uint8")
    true_label = class_names[label_batch[idx]]
    
    # Hacer diagnóstico
    result = diagnose_plant(model, image_batch[idx], class_names)
    
    # Visualizar
    axes[idx].imshow(image)
    axes[idx].axis('off')
    
    # Determinar si es correcto
    is_correct = result['predicted_class'] == true_label
    color = '#2ecc71' if is_correct else '#e74c3c'
    status = '✅ CORRECTO' if is_correct else '❌ INCORRECTO'
    
    title = f"{status}\n"
    title += f"Predicción: {result['predicted_class'].replace('___', ' - ').replace('_', ' ')}\n"
    title += f"Confianza: {result['confidence']:.1%}\n"
    title += f"Real: {true_label.replace('___', ' - ').replace('_', ' ')}"
    
    axes[idx].set_title(title, fontsize=10, fontweight='bold', 
                        color=color, pad=10)

plt.suptitle('🌱 AGRO-AI VISION PRO - Resultados de Diagnóstico', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('demo_predictions.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Demo guardado como 'demo_predictions.png'")

# ==========================================
# 💾 GUARDAR MODELO Y METADATOS
# ==========================================
print("\n💾 Guardando modelo y metadatos...")

# Guardar modelo
model.save('agro_ai_vision_pro.keras')
print("✅ Modelo guardado como 'agro_ai_vision_pro.keras'")

# Guardar metadatos
metadata = {
    'model_name': 'AGRO-AI VISION PRO',
    'version': '1.0.0',
    'created_at': datetime.now().isoformat(),
    'architecture': 'MobileNetV2 + Transfer Learning',
    'image_size': [IMG_HEIGHT, IMG_WIDTH],
    'num_classes': num_classes,
    'class_names': class_names,
    'final_accuracy': float(history['val_accuracy'][-1]),
    'final_loss': float(history['val_loss'][-1]),
    'best_accuracy': float(max(history['val_accuracy'])),
    'epochs_trained': len(history['accuracy']),
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
}

with open('model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("✅ Metadatos guardados como 'model_metadata.json'")

# ==========================================
# 📈 REPORTE FINAL
# ==========================================
print("\n" + "="*60)
print("📈 REPORTE FINAL DEL SISTEMA")
print("="*60)

print(f"""
🎯 MÉTRICAS DE RENDIMIENTO:
   • Accuracy Final (Validación): {metadata['final_accuracy']:.2%}
   • Mejor Accuracy:              {metadata['best_accuracy']:.2%}
   • Loss Final:                  {metadata['final_loss']:.4f}
   • Épocas Entrenadas:           {metadata['epochs_trained']}

🧠 ARQUITECTURA:
   • Modelo Base:                 {metadata['architecture']}
   • Tamaño de Imagen:            {IMG_HEIGHT}x{IMG_WIDTH}
   • Clases Detectables:          {num_classes}
   • Parámetros Totales:          {model.count_params():,}

📊 CAPACIDADES:
   • Transfer Learning con ImageNet
   • Data Augmentation avanzado
   • Fine-tuning adaptativo
   • Predicciones con Top-K
   • Matriz de confusión
   • Visualizaciones profesionales

💾 ARCHIVOS GENERADOS:
   • agro_ai_vision_pro.keras     (Modelo entrenado)
   • model_metadata.json          (Metadatos)
   • training_history.png         (Gráficos de entrenamiento)
   • confusion_matrix.png         (Matriz de confusión)
   • demo_predictions.png         (Demo de predicciones)
   • best_model.keras             (Mejor checkpoint)

🚀 LISTO PARA PRODUCCIÓN!
""")

print("="*60)
print("✅ Sistema AGRO-AI VISION PRO completamente entrenado!")
print("="*60)

# ==========================================
# 📥 DESCARGAR ARCHIVOS
# ==========================================
print("\n📥 ¿descargar los archivos generados? (y/n)")
download = input()

if download.lower() == 'y':
    files.download('agro_ai_vision_pro.keras')
    files.download('model_metadata.json')
    files.download('training_history.png')
    files.download('confusion_matrix.png')
    files.download('demo_predictions.png')
    print("✅ Archivos descargados exitosamente!")
else:
    print("ℹ️ Puede descargar los archivos manualmente desde el panel de archivos")

print("\n🎉 ¡Gracias por usar AGRO-AI VISION PRO!")
